# Device Actions

Generates datasets for performing actions on devices in a synthetic home.

In [11]:
import openai

from home_assistant_datasets.secrets import get_secret
from home_assistant_datasets import model_client

MODEL_ID = "gpt-3.5-turbo-0125"

openai = openai.OpenAI(api_key=get_secret("openai_api_key"))
model = model_client.ModelClient(openai, MODEL_ID)

# Generation

In [5]:
import pathlib
import yaml


DATASET_DIR = pathlib.Path("../datasets/")
DEVICES_DIR = DATASET_DIR / "devices"
SEEDS_DIR = pathlib.Path("./seeds")
SEED_DEVICE_ACTIONS_FILE = SEEDS_DIR / "device-actions.yaml"

with open(SEED_DEVICE_ACTIONS_FILE) as f:
    seed_device_actions = list(yaml.load_all(f.read(), Loader=yaml.Loader))

seed_devices_prompt = yaml.dump(seed_device_actions, sort_keys=False, explicit_start=True)
print(seed_devices_prompt)


---
- home: mountain-cabin-us
  device:
    name: Kitchen Light
    area: Kitchen
    device_type: light
  actions:
  - sentences:
    - Turn on the kitchen light
    - Kitchen light on
    - Please turn on the kitchen light
    - Kitchen light on please
    state: false
    expected_state: true
  - sentences:
    - Turn off the kitchen light
    - Kitchen light off
    - Please turn off the kitchen light
    - Kitchen light off please
    state: false
    expected_state: true



In [6]:
SUMMARY_PROMPT = f"""
You are an expert Smart Home agent who can evaluate the performance of a smart
home, and perform useful actions on behalf of a user.

A device in Home Assistant represents a physical or virtual object, represented
by different entities. A device has attributes for its configuration and state,
for example a thermostat may have a mode attribute, or target or current temperature
attributes.

You generate a simple evaluation dataset for home data. The input dataset
contains the home, description information like location, areas, and devices.
The output data are actions a user may ask to take on a devie.

You answer in yaml, and here are examples:

{seed_devices_prompt}
"""

In [14]:
import itertools
import random
from tqdm.auto import tqdm
import shutil

N_DATAPOINTS = 40

homes = []
for path in DEVICES_DIR.glob("*.yaml"):
    with path.open("r") as f:
        content = f.read()
    home_id = path.name.split(".")[0]  # Strip the .yaml extension
    home_data = yaml.load(content, Loader=yaml.Loader)
    homes.append((home_id, home_data))

DEVICE_ACTIONS_OUTPUT_DIR = DATASET_DIR / "device-actions"

# Wipe existing summaries
shutil.rmtree(DEVICE_ACTIONS_OUTPUT_DIR, ignore_errors=True)
DEVICE_ACTIONS_OUTPUT_DIR.mkdir(exist_ok=True)

random.shuffle(homes)
if len(homes) > N_DATAPOINTS:
    homes = homes[:N_DATAPOINTS]

skipped = 0

with tqdm(total=len(homes)) as pbar:
    for home_id, home in homes:
        with open(DEVICE_ACTIONS_OUTPUT_DIR / f"{home_id}.yaml", "w") as summary_output:
            home_yaml = yaml.dump(home, sort_keys=False)

            home_yaml =
            print(home_yaml)
            prompt = f"Generate a few device actions for each device for this home in a yaml.\n\n{home_yaml}\n\Output\n---"
            response_obj = None
            for i in range(3):
                response = model.complete(SUMMARY_PROMPT, prompt)
                try:
                    response_obj = yaml.safe_load(response)
                except yaml.YAMLError:
                    skipped += 1
                    continue
            if response_obj is not None:
                updated_home = home.copy()
                updated_home.update({"summaries": response_obj})
                summary_output.write(yaml.dump(updated_home, explicit_start=True, sort_keys=False))
            pbar.set_description(f"Skipped {skipped}")
            pbar.update(1)

<>:33: SyntaxWarning: invalid escape sequence '\O'
<>:33: SyntaxWarning: invalid escape sequence '\O'
/tmp/ipykernel_1159289/722108639.py:33: SyntaxWarning: invalid escape sequence '\O'
  prompt = f"Generate a few device actions for each device for this home in a yaml.\n\n{home_yaml}\n\Output\n---"
  0%|          | 0/40 [00:00<?, ?it/s]

name: Chalet Montagne
country_code: FR
location: Alpine village in the French Alps
type: Ski chalet
amenities:
- "5 chambres avec chemin\xE9e en pierre"
- 3 salles de bains avec sauna
- Vue panoramique sur les montagnes
- Salon chaleureux avec poutres apparentes
- Cuisine rustique avec coin repas
- "Terrasse en bois avec jacuzzi ext\xE9rieur"
- "Local \xE0 skis chauff\xE9"
areas:
- Chambre 1
- Chambre 2
- Chambre 3
- Chambre 4
- Chambre 5
- Salon
- Cuisine
- "Salle \xE0 manger"
- Salle de bain 1
- Salle de bain 2
- Salle de bain 3
- Sauna
- Terrasse
- Jacuzzi
- "Local \xE0 skis"
devices:
  Chambre 1:
  - name: Lampe
    device_type: light
    device_info:
      model: "Ampoule LED Connect\xE9e"
      manufacturer: LIFX
      sw_version: 5.1.7
  Chambre 2:
  - name: Lampe
    device_type: light
    device_info:
      model: "Ampoule LED Connect\xE9e"
      manufacturer: LIFX
      sw_version: 5.1.7
  Chambre 3:
  - name: Lampe
    device_type: light
    device_info:
      model: "Ampoul

  0%|          | 0/40 [00:02<?, ?it/s]
/tmp/ipykernel_1159289/722108639.py:33: SyntaxWarning: invalid escape sequence '\O'
  prompt = f"Generate a few device actions for each device for this home in a yaml.\n\n{home_yaml}\n\Output\n---"


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}